In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
# setting path
sys.path.append("/content/drive/MyDrive/simclr")
sys.path.append("/content/drive/MyDrive/simclr/layers")
sys.path.append("/content/drive/MyDrive/simclr/models")
sys.path.append("/content/drive/MyDrive/simclr/loss")
sys.path.append("/content/drive/MyDrive/simclr/optim")
sys.path.append("/content/drive/MyDrive/simclr/dataloader")
sys.path.append("/content/drive/MyDrive/simclr/instance-discrim")
sys.path.append("/content/drive/MyDrive/simclr/dloader")

In [3]:
from simclr import SimClr
from ntxent import nt_xent_loss
from ntxentgit import SimCLR_Loss
from augment import TransformsSimCLR
from utils import *
from LARS import LARS
from downstream import DownStream
from dataloader import *
from ResNetCifar import ResNetCifar
from utilsInstance import *
from NCEAverage import *
from NCECriterion import *
from alias_multinomial import *
from Cifar10Instance import *
from downstreamnew import DownStreamNew

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pickle

In [5]:
shuffle = True
train_loader_ds, val_loader_ds = train_loader_dstream("CIFAR10",256,0.1,shuffle)

Files already downloaded and verified


In [7]:
model = SimClr('resnet50',64).cuda()
model.load_state_dict(torch.load("/content/drive/MyDrive/simclr/saved-models/CIFAR10-RES50-SIMCLR-BS128-PD64-ADAM"))

<All keys matched successfully>

In [8]:
model_ds = DownStreamNew(model.encoder,model.projector.layers[:2],2048,10).cuda()
optimizer = torch.optim.Adam(model_ds.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
epochs = 100
model_ds,train_loss_list,train_acc_list,val_loss_list,val_acc_list = train_ds(train_loader_ds, val_loader_ds, model_ds, criterion, optimizer,epochs,True,path="/content/drive/MyDrive/simclr/dstream-saved-models/CIFAR10-RES50-SIMCLR-BS128-PD64-ADAM")

Epoch: 0, step: 50/175, training_loss: 0.962407648563385, training_acc: 0.7265625
Epoch: 0, step: 100/175, training_loss: 1.0222973823547363, training_acc: 0.7109375
Epoch: 0, step: 150/175, training_loss: 0.8550984859466553, training_acc: 0.69921875
Epoch: 0 completed, average train loss: 1.1540500715800694, average training_acc: 0.6944419642857143, average validation loss: 0.751898047171141, average validation_acc: 0.7417763157894737 time taken: 0.4290411114692688 mins
Epoch: 1, step: 50/175, training_loss: 0.7191250324249268, training_acc: 0.76171875
Epoch: 1, step: 100/175, training_loss: 0.7134823799133301, training_acc: 0.7734375
Epoch: 1, step: 150/175, training_loss: 0.8863956928253174, training_acc: 0.75390625
Epoch: 1 completed, average train loss: 0.7606339793545859, average training_acc: 0.7449107142857143, average validation loss: 0.7261111014767697, average validation_acc: 0.7532894736842105 time taken: 0.4284168402353922 mins
Epoch: 2, step: 50/175, training_loss: 0.7405

In [9]:
with open("/content/drive/MyDrive/simclr/dstream-trainloss-logs/CIFAR10-RES50-SIMCLR-BS128-PD64-ADAM", "wb") as fp:   #Pickling
    pickle.dump(train_loss_list, fp)
with open("/content/drive/MyDrive/simclr/dstream-trainacc-logs/CIFAR10-RES50-SIMCLR-BS128-PD64-ADAM", "wb") as fp:   #Pickling
    pickle.dump(train_acc_list, fp)
with open("/content/drive/MyDrive/simclr/dstream-valloss-logs/CIFAR10-RES50-SIMCLR-BS128-PD64-ADAM", "wb") as fp:   #Pickling
    pickle.dump(val_loss_list, fp)
with open("/content/drive/MyDrive/simclr/dstream-valacc-logs/CIFAR10-RES50-SIMCLR-BS128-PD64-ADAM", "wb") as fp:   #Pickling
    pickle.dump(val_acc_list, fp)

In [10]:
test_loader = test_loader("CIFAR10",128)

Files already downloaded and verified


In [11]:
test_ds(model_ds,test_loader)

Test accuracy: 0.7540064102564102, time taken: 0.08241377274195354
